# Performing Testing at Each Round of Training 

Use for developing (autoreloads changes made across packages)

In [1]:
%load_ext autoreload
%autoreload 2

## Start the network
Before running this notebook, start the network with `./scripts/fedbiomed_run network`

## Setting the node up
It is necessary to previously configure a node:
1. `./scripts/fedbiomed_run node add`
  * Select option 2 (default) to add MNIST to the node
  * Confirm default tags by hitting "y" and ENTER
  * Pick the folder where MNIST is downloaded (this is due torch issue https://github.com/pytorch/vision/issues/3549)
  * Data must have been added (if you get a warning saying that data must be unique is because it's been already added)
  
2. Check that your data has been added by executing `./scripts/fedbiomed_run node list`
3. Run the node using `./scripts/fedbiomed_run node run`. Wait until you get `Starting task manager`. it means you are online.

## Define an experiment model and parameters"

Declare a torch.nn MyTrainingPlan class to send for training on the node

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlan(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlan, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss


This group of arguments correspond respectively:
* `model_args`: a dictionary with the arguments related to the model (e.g. number of layers, features, etc.). This will be passed to the model class on the node side.
* `training_args`: a dictionary containing the arguments for the training routine (e.g. batch size, learning rate, epochs, etc.). This will be passed to the routine on the node side.

**NOTE:** typos and/or lack of positional (required) arguments will raise error. 🤓

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

## Declare and run the experiment

- search nodes serving data for these `tags`, optionally filter on a list of node ID with `nodes`
- run a round of local training on nodes with model defined in `model_path` + federation with `aggregator`
- run for `round_limit` rounds, applying the `node_selection_strategy` between the rounds

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlan,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-25 08:45:58,488 fedbiomed INFO - Component environment:
2022-03-25 08:45:58,489 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-25 08:45:58,687 fedbiomed INFO - Messaging researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7fe2cf1439a0>
2022-03-25 08:45:58,750 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-25 08:46:08,761 fedbiomed INFO - Node selected for training -> node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c
2022-03-25 08:46:08,813 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0003/my_model_acbc9355-8108-419f-8a33-b0d45cf06c97.py
2022-03-25 08:46:08,944 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0003/my_model_acbc9355-8108-419f-8a33-b0d45cf06c97.py successful, with status code 201
2022-03-25 08:46:09,210 fedbi

Let's start the experiment.

By default, this function doesn't stop until all the `round_limit` rounds are done for all the nodes

In [4]:
exp.run()

2022-03-25 08:46:09,216 fedbiomed INFO - Sampled nodes in round 0 ['node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c']
2022-03-25 08:46:09,217 fedbiomed INFO - Send message to node node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - {'researcher_id': 'researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849', 'job_id': 'ef07c9fc-44c7-41b2-a120-f0c6ac373648', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_acbc9355-8108-419f-8a33-b0d45cf06c97.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_init_e73ea983-0b4f-43ee-8b7c-e6137f324620.pt', 'model_class': 'MyTrainingPlan', 'training_data': {'node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c': ['dataset_2af1ef7b-d8cf-4504-8315-9c681b222

2022-03-25 08:46:43,751 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.096692 
					 ---------
2022-03-25 08:46:44,170 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.334916 
					 ---------
2022-03-25 08:46:50,392 fedbiomed INFO - TESTING ON GLOBAL PARAMETERS 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Completed: 18000/18000 (100%) 
 					 ACCURACY: 0.963056 
					 ---------
2022-03-25 08:46:50,545 fedbiomed INFO - INFO FROM NODE node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c
MESSAGE: 
results uploaded successfully 
----------------------------------------
2022-03-25 08:46:59,467 fedbiomed INFO - Downloading model params after training on node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - from http://localhost:8844/media/uploads/2022/03/25/node_params_fc8c24ef-0a81-45fd-a250-cb8e472a

2

In [5]:
exp.run(rounds=8, increase=True)

2022-03-25 08:46:59,690 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 10
2022-03-25 08:46:59,692 fedbiomed INFO - Sampled nodes in round 2 ['node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c']
2022-03-25 08:46:59,693 fedbiomed INFO - Send message to node node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - {'researcher_id': 'researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849', 'job_id': 'ef07c9fc-44c7-41b2-a120-f0c6ac373648', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_acbc9355-8108-419f-8a33-b0d45cf06c97.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_7f031e6c-9213-4448-8ec5-c285496702fb.pt', 'model_class': 'MyTrainingPlan', 'training_da


--------------------
Fed-BioMed researcher stopped due to exception:
FB407: list of nodes became empty when training
--------------------


2022-03-25 08:47:15,693 fedbiomed INFO - TESTING ON GLOBAL PARAMETERS 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Completed: 18000/18000 (100%) 
 					 ACCURACY: 0.964444 
					 ---------
2022-03-25 08:47:16,310 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 480/42000 (1%) 
 					 Loss: 0.206689 
					 ---------
2022-03-25 08:47:17,014 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 960/42000 (2%) 
 					 Loss: 0.349654 
					 ---------
2022-03-25 08:47:17,652 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 1440/42000 (3%) 
 					 Loss: 0.109764 
					 ---------
2022-03-25 08:47:18,459 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 1920/42000 (5%) 
 					 Loss: 0.160588 
					 ---------
2022-03-25 08:47:19,1

Local training results for each round and each node are available via `exp.training_replies()` (index 0 to (`rounds` - 1) ).

For example you can view the training results for the last round below.

Different timings (in seconds) are reported for each dataset of a node participating in a round :
- `rtime_training` real time (clock time) spent in the training function on the node
- `ptime_training` process time (user and system CPU) spent in the training function on the node
- `rtime_total` real time (clock time) spent in the researcher between sending the request and handling the response, at the `Job()` layer

In [5]:
print("\nList the training rounds : ", exp.training_replies().keys())

print("\nList the nodes for the last training round and their timings : ")
round_data = exp.training_replies()[rounds - 1].data()
for c in range(len(round_data)):
    print("\t- {id} :\
    \n\t\trtime_training={rtraining:.2f} seconds\
    \n\t\tptime_training={ptraining:.2f} seconds\
    \n\t\trtime_total={rtotal:.2f} seconds".format(id = round_data[c]['node_id'],
        rtraining = round_data[c]['timing']['rtime_training'],
        ptraining = round_data[c]['timing']['ptime_training'],
        rtotal = round_data[c]['timing']['rtime_total']))
print('\n')
    
exp.training_replies()[rounds - 1].dataframe()


List the training rounds :  dict_keys([0, 1])

List the nodes for the last training round and their timings : 
	- node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c :    
		rtime_training=4.37 seconds    
		ptime_training=17.36 seconds    
		rtime_total=25.03 seconds




,success,msg,dataset_id,node_id,params_path,params,timing
0,True,,dataset_2af1ef7b-d8cf-4504-8315-9c681b222edd,node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c,/home/ybouilla/fedbiomed/var/experiments/Exper...,"{'conv1.weight': [[tensor([[-0.2735, 0.0247, ...","{'rtime_training': 4.370531685999595, 'ptime_t..."


Federated parameters for each round are available via `exp.aggregated_params()` (index 0 to (`rounds` - 1) ).

For example you can view the federated parameters for the last round of the experiment :

In [6]:
print("\nList the training rounds : ", exp.aggregated_params().keys())

print("\nAccess the federated params for the last training round :")
print("\t- params_path: ", exp.aggregated_params()[rounds - 1]['params_path'])
print("\t- parameter data: ", exp.aggregated_params()[rounds - 1]['params'].keys())



List the training rounds :  dict_keys([0, 1])

Access the federated params for the last training round :
	- params_path:  /home/ybouilla/fedbiomed/var/experiments/Experiment_0000/aggregated_params_a0accb65-834e-4d2f-b748-a2360b0c5b0a.pt
	- parameter data:  odict_keys(['conv1.weight', 'conv1.bias', 'conv2.weight', 'conv2.bias', 'fc1.weight', 'fc1.bias', 'fc2.weight', 'fc2.bias'])


Feel free to run other sample notebooks or try your own models :D

## Testing using your own testing metric

In [1]:
import torch
import torch.nn as nn
from fedbiomed.common.training_plans import TorchTrainingPlan
from fedbiomed.common.data import DataManager
from torchvision import datasets, transforms

# Here we define the model to be used. 
# You can use any class name (here 'Net')
class MyTrainingPlanCM(TorchTrainingPlan):
    def __init__(self, model_args: dict = {}):
        super(MyTrainingPlanCM, self).__init__(model_args)
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        
        # Here we define the custom dependencies that will be needed by our custom Dataloader
        # In this case, we need the torch DataLoader classes
        # Since we will train on MNIST, we need datasets and transform from torchvision
        deps = ["from torchvision import datasets, transforms"]
        
        self.add_dependency(deps)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        
        
        output = F.log_softmax(x, dim=1)
        return output

    def training_data(self, batch_size = 48):
        # Custom torch Dataloader for MNIST data
        transform = transforms.Compose([transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))])
        dataset1 = datasets.MNIST(self.dataset_path, train=True, download=False, transform=transform)
        train_kwargs = {'batch_size': batch_size, 'shuffle': True}
        return DataManager(dataset=dataset1, **train_kwargs)
    
    def training_step(self, data, target):
        output = self.forward(data)
        loss   = torch.nn.functional.nll_loss(output, target)
        return loss

    def testing_step(self, data, target):
        
        output = self.forward(data)
        loss1   = torch.nn.functional.nll_loss(output, target)
        output = self(data)
        loss2   = torch.nn.functional.nll_loss(output, target)
        return [loss1, loss2]

In [2]:
model_args = {}

training_args = {
    'batch_size': 48, 
    'lr': 1e-3, 
    'epochs': 1, 
    'dry_run': False,  
    'batch_maxnum': 100, # Fast pass for development : only use ( batch_maxnum * batch_size ) samples
    'test_ratio': .3,
    'test_on_local_updates': True, 
    'test_on_global_updates': True
}

In [3]:
from fedbiomed.researcher.experiment import Experiment
from fedbiomed.researcher.aggregators.fedavg import FedAverage

tags =  ['#MNIST', '#dataset']
rounds = 2

exp = Experiment(tags=tags,
                 model_args=model_args,
                 model_class=MyTrainingPlanCM,
                 training_args=training_args,
                 round_limit=rounds,
                 aggregator=FedAverage(),
                 node_selection_strategy=None)

2022-03-25 08:50:26,236 fedbiomed INFO - Component environment:
2022-03-25 08:50:26,237 fedbiomed INFO - type = ComponentType.RESEARCHER
2022-03-25 08:50:26,395 fedbiomed INFO - Messaging researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849 successfully connected to the message broker, object = <fedbiomed.common.messaging.Messaging object at 0x7f329d5a8eb0>
2022-03-25 08:50:26,438 fedbiomed INFO - Searching dataset with data tags: ['#MNIST', '#dataset'] for all nodes
2022-03-25 08:50:36,459 fedbiomed INFO - Node selected for training -> node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c
2022-03-25 08:50:36,484 fedbiomed DEBUG - Model file has been saved: /home/ybouilla/fedbiomed/var/experiments/Experiment_0004/my_model_4eebc9c5-fc0f-43ba-a050-bf8aa65de6ee.py
2022-03-25 08:50:36,511 fedbiomed DEBUG - upload (HTTP POST request) of file /home/ybouilla/fedbiomed/var/experiments/Experiment_0004/my_model_4eebc9c5-fc0f-43ba-a050-bf8aa65de6ee.py successful, with status code 201
2022-03-25 08:50:36,651 fedbi

In [4]:
exp.run()

2022-03-25 08:50:36,657 fedbiomed INFO - Sampled nodes in round 0 ['node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c']
2022-03-25 08:50:36,658 fedbiomed INFO - Send message to node node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - {'researcher_id': 'researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849', 'job_id': '9674db17-36f7-4667-9c27-f2f166333ffa', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_4eebc9c5-fc0f-43ba-a050-bf8aa65de6ee.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_init_d9e88ed7-46ae-4124-b52f-d636a2e7fb96.pt', 'model_class': 'MyTrainingPlanCM', 'training_data': {'node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c': ['dataset_2af1ef7b-d8cf-4504-8315-9c681b2

2022-03-25 08:51:26,208 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.111478 
					 ---------
2022-03-25 08:51:26,649 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 4320/42000 (10%) 
 					 Loss: 0.087185 
					 ---------
2022-03-25 08:51:38,196 fedbiomed INFO - TESTING ON GLOBAL PARAMETERS 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Completed: 18000/18000 (100%) 
 					 Custom_1: 0.126069 
					 Custom_2: 0.126069 
					 ---------
2022-03-25 08:51:38,358 fedbiomed INFO - INFO FROM NODE node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c
MESSAGE: 
results uploaded successfully 
----------------------------------------
2022-03-25 08:51:46,926 fedbiomed INFO - Downloading model params after training on node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - from http://localhost:8844/media/uploads/2022/03/25/node_params_1d7dbd

2

In [ ]:
exp.run(rounds=8, increase=True)

2022-03-25 08:51:47,137 fedbiomed DEBUG - Auto increasing total rounds for experiment from 2 to 10
2022-03-25 08:51:47,141 fedbiomed INFO - Sampled nodes in round 2 ['node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c']
2022-03-25 08:51:47,142 fedbiomed INFO - Send message to node node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c - {'researcher_id': 'researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849', 'job_id': '9674db17-36f7-4667-9c27-f2f166333ffa', 'training_args': {'test_ratio': 0.3, 'test_on_local_updates': True, 'test_on_global_updates': True, 'test_metric': None, 'test_metric_args': {}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}, 'training': True, 'model_args': {}, 'command': 'train', 'model_url': 'http://localhost:8844/media/uploads/2022/03/25/my_model_4eebc9c5-fc0f-43ba-a050-bf8aa65de6ee.py', 'params_url': 'http://localhost:8844/media/uploads/2022/03/25/aggregated_params_0497cc38-0737-4263-8124-6e17d9c8f6c0.pt', 'model_class': 'MyTrainingPlanCM', 'training_

2022-03-25 08:52:50,694 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 1920/42000 (5%) 
 					 Loss: 0.304058 
					 ---------
2022-03-25 08:52:51,123 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 2400/42000 (6%) 
 					 Loss: 0.036720 
					 ---------
2022-03-25 08:52:51,549 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 2880/42000 (7%) 
 					 Loss: 0.015316 
					 ---------
2022-03-25 08:52:51,976 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 3360/42000 (8%) 
 					 Loss: 0.178931 
					 ---------
2022-03-25 08:52:52,404 fedbiomed INFO - TRAINING 
					 NODE_ID: node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c 
					 Epoch: 1 | Completed: 3840/42000 (9%) 
 					 Loss: 0.082312 
					 ---------
2022-03-25 08:52:52,837 fedbiomed I

2022-03-25 08:53:47,845 fedbiomed DEBUG - researcher_8b7f6b87-cc47-4ec3-bd46-4665fa573849
2022-03-25 08:53:47,917 fedbiomed INFO - INFO FROM NODE node_bd747f4b-d47a-4172-bd61-18e0ca3a2a9c
MESSAGE: 
training with arguments {'history_monitor': <fedbiomed.node.history_monitor.HistoryMonitor object at 0x7f17394548e0>, 'node_args': {'gpu': False, 'gpu_num': None, 'gpu_only': False}, 'batch_size': 48, 'lr': 0.001, 'epochs': 1, 'dry_run': False, 'batch_maxnum': 100}
----------------------------------------
